# Explanation

I've included these 2 resources to provide context on the original challenge and framing of robotic control from the perspective of original classical control approaches.

Robotic control aims at creating policies to take high level goals and intentions based on the robots task and planning system and convert them into specific actions in the environment to accomplish these goals. This is the most complex part of the robot.

For several decades, most robotic control systems were built on man-made analytical models meant to capture the kinematics and dynamics of specific physical systems.

Specifically, researchers tried to directly capture the equations governing the rigid body kinematics and dynamics of robotic systems and use these to manually dictate robot movement based on these physics models.

My goal in including these resources is not to introduce any specific physics models, as they often require large time investment to understand are now almost entirely obsolete. Instead, I find it valuable to frame robotic control in the original complexity of manual attempts at analytical models.

This helps to appreciate the difficulty of the problem implied by Moravec's paradox, and resembles the original use of heavy feature engineering in early machine learning which later got completely replaced by deep learning. A similar phenomenon has occurred in robotics.

# Notes: Classical Control

Notes on the relevant chapters of _Modern Robotics_, skipping over the extensive chapters on the specifics of forward and inverse kinematics since we care more about an understanding of how they influence control systems, rather than the specific physics equations used (which are now obsolete).

### Chapter 1: Preview

The goal of robotics is to create intelligent machines that can behave and think like humans.

Robotics is about turning ideas into action.

> Robots turn abstract goals into physical action.
> 

A robot is made of:

- A group of **rigid bodies**
- Connected by **joints**
- Driven by **actuators**

The links may not be completely rigid, and the joints may be affected by elasticity, backlash, friction, and hysteresis

Modern robotic links are driven primarily by AC/DC motor actuators, stepper motors, shape memory alloys, or pneumatics/hydraulics.

We need lightweight, low rotational speeds, and high force/torque actuators.

Current motors are low torque/high rotation so we need transformation systems.

These are usually gears/cable drives/pulleys. These need to have low slippage and backlash.

Sensors are used on the robot to measure displacement, and sometimes velocity. Forces and torques can also be measured. Vision-only cameras and other sensors can also be used..

A rigid body in 3d has 6 DoF (3 for position, 3 for orientation).

We need to understand the configuration of a robot, made of the 6 specifications for each rigid body.

A robot has some part meant to interact with the world called the **end-effector**.

The tasks space is the part of the configuration space that the end-effector can reach.

Rigid-body motions (chapter 3) focuses on how to mathematically describe a rigid-body moving in 3d space. Uses the **exponential coordinate** representation.

Forward kinematics (chapter 4) deals with predicting the configuration of the end-effector based on the joint positions. Uses the **product of exponentials (PoE).**

Velocity kinematics and statics (chapter 5) uses the Jacobian of forward kinematics to predict the relationship between joint linear/angular velocities and those of the end-effector, and limitations on movement (kinematic singularities) based on the Jacobian failing to have maximal rank.

Inverse kinematics (chapter 6) is trying to determine the joint positions to achieve a desired end-effector configuration.

Kinematics of closed chains (chapter 7) has more forward and inverse kinematics solutions and is more complex.

Dynamics of open chains (chapter 8) studies the forward dynamics problem of determining joint accelerations from forces/torques, and inverse dynamics to determine necessary join forces/torques to create end-effector acceleration.

Trajectory generation (chapter 9) shows how robots differ from automated machines by being able to perform different tasks. Covers the automatic generation of joint trajectories given task input data. Consists of a **path** of configurations and a **time series** of when those configurations are achieved. Focuses on (1) point-to-point straight line trajectories in joint/task space (2) smooth trajectories passing through via points (3) time-optimal trajectories along specific paths.

Motion planning (chapter 10) is about avoiding collusions in a cluttered workspace, avoiding joint/actuator limits, and other physical constraints. Path planning is a subset of this problem.

Robot control (chapter 11) involves robots using motion control, force control, hybrid motion-force control, and impedance control to convert task specification to forces/torques at actuators. Practical control systems compensate for uncertainties caused by friction, elasticity, backlash, and hysteresis with feedback control. They combine feedback control and approximate dynamic modeling into computed torque control.

Grasping and manipulation (chapter 12) focuses on choosing contacts to immobilize an object with form closure and force closure, and broadly using contact modeling for grasping and other problems.

Wheeled mobile robots (chapter 13) are covered in the final chapter.

### Chapter 2: Configuration Space

A robot is a group of rigid bodies called **links** connected with **joints** using **actuators** to apply force and torque.

The **configuration** of a robot defines where it is.

The **degrees of freedom** is the minimum number of variables needed to represent it’s configuration.

The configuration space (C-space) is the space of all possible configurations.

$$
\textrm{DoF} = \textrm{(sum of freedoms of points)} - \textrm{(num indp. constraints)} \\
\textrm{DoF} = \textrm{(num vars) - (num independent equations)}
$$

A rigid body moving in 3d (a **spatial rigid body**) has 6 degrees of freedom (3 coordinates of A, two angles to B, 1 angle to C). Planar rigid body has 3 DoF.

$$
\mathrm{DoF} = \textrm{(sum of freedoms of bodies) - (num indp. constraints)}
$$

**Grübler’s formula** gives a way to determine the number of degrees of freedom of planar/spatial robots by counting DoF and constraints of joints.

The main joints have 1 dof:

- Revolute joints allow rotational motion
- Prismatic joints allow translational motion
- Helical joints use both at once

Some joints can have many dof, like cylindrical joint, universal joint, spherical joint.

![Screenshot 2024-10-29 at 1.12.39 PM.png](../../images/Screenshot_2024-10-29_at_1.12.39_PM.png)

Grubler’s formula for the degrees of freedom of a robot:

$$
\textrm{dof} = m(N - 1) - \sum_{i=1}^{J} c_i = m(N-1-J) + \sum_{i=1}^J f_i
$$

- left part is rigid body dofs. we don’t count the first one because it’s grounded.
- right part is subtracting the constraints of each joint.
- basically, this is just adding up all the joint dofs.

Open-chain robots have a loop, closed chain robots do not.

![Screenshot 2024-10-29 at 2.11.43 PM.png](../../images/Screenshot_2024-10-29_at_2.11.43_PM.png)

Each space has a shape known as its **topology**. C-space can be written as the product of smaller spaces.

- The C-space of a planar rigid body is $\mathbb{R}^2 \times \mathbb{S}^1$
- The C-space of rigid body in 3d is $\mathbb{R}^3 \times \mathbb{S}^2 \times \mathbb{S}^1$

We need a **representation** of a C-space in order to perform computations. There are many valid representations. The representation is a choice, whereas the topology is fundamental.

Equivalent topologies are ones that can be morphed into each other without cutting or gluing. This basically means that units can be squished into each other in terms of transformations.

Choice of **parameterization** can lead to irregularities/singularities in the C-space even though the actual topology doesn’t have them - for example the singularities at the north and south poles of a sphere when parameterizing them with latitude and longitude.

Explicit parameterization can have singularities.

Implicit parameterization uses a higher dimensional space to eliminate singularities, though it requires more variables than dof.

**Holonomic constraints** reduce the dimensions of a C-space. Velocity constraints in a matrix are **Pfaffian constraints**. Pfaffian constraints that are nonintegrable are **nonholonomic constraints**.

The **task space** is the space where the robot’s task is naturally expressed, defined differently for each problem.

The **workspace** is the C-space of the end-effector.

A point in the task space may be achievable by more than one robot configuration. All points in the task space are reachable by at least one configuration.

### Chapter 11: Robot Control

Robots can use different strategies to achieve the behaviors necessary for different tasks:

- **Motion control** - the robot acts as a source of motion for tasks like moving objects.
- **Force control** - the robot acts as a source of force like pushing a polishing wheel against something.
- **Hybrid motion-force control** - the robot may need to maintain motions in one direction and forces in another; ex: controlling chalk where you have force control in the direction normal to the board and motion control in the plane parallel to the board.
- **Impedance control** - the robot can be a damper/spring/mass yielding in response to forces applied to it.

Motion control and force control can never be used in the same direction:

- Motion control means the robot is controlling it’s end-effector position. Then the environment determines the forces acting on it.
- Force control means the robot is controlling its’ end-effector force/torque. Then the environment determines the position change caused.

> Once we have a control goal for the task, we can use feedback control to achieve it. 

Feedback control uses position, velocity, and force sensors to measure the actual behavior of the robot, compares it with the desired behavior, and modulates the control signals sent to the actuators.
> 

The typical sensors used for control:

- Joint position/angle sensing - potentiometers, encoders, resolvers
- Joint velocity sensing - tachometers
- Joint force-torque sensors
- Multi-axis force-torque sensors at the “wrist” between the end of the arm and end effectors

The controller checks sensors and updates actuator signals at hundreds to thousands of Hz.

In an ideal system, can assume that sensors get perfect position/velocity/force/torque data, and that the amplifier, actuator, transmission systems perfectly convert signals into forces/torques.

A diagram of the control system without disturbance forces from the environment factored in.

![Screenshot 2024-10-30 at 2.48.57 PM.png](../../images/Screenshot_2024-10-30_at_2.48.57_PM.png)

A simplified diagram assuming perfect signal to torque/force conversion.

![Screenshot 2024-10-30 at 2.49.13 PM.png](../../images/Screenshot_2024-10-30_at_2.49.13_PM.png)

> The differential equation governing the evolution of the joint error $\theta_e (t)$ of the controlled system is called the **error dynamics.** The purpose of the feedback controller is to create an error dynamics such that $\theta_e (t)$ tends to zero, or a small value, as $t$ increases.
> 

The ideal controller drives the error to 0 instantly and keeps it there forever. Practically, error takes time to reduce, and may oscillate around the perfect solution.

Error response can be described by **transient response** and **steady-state response**. **Steady-state error** is the asymptotic error. Overshoot is when error goes past the target, and settling time is when error reaches <2% threshold.

The feedforward-feedback controller uses feedforward control to determine motion even with no error and feedback control which limits the accumulation of error. 

The **proportional-integral-derivative control (PID)** is a popular control mechanism.

![Screenshot 2024-10-30 at 3.03.10 PM.png](../../images/Screenshot_2024-10-30_at_3.03.10_PM.png)

In ideal motion control, motion is unaffected by disturbance forces.

In ideal force control, the force applied to the end-effector is unaffected by disturbance motions to the end-effector.

### Chapter 12: Grasping and Manipulation

Chapter 11 covers force control, motion control, hybrid motion-force control, and impedance control are the first dip into the focus on manipulating objects in the environment, rather than operating the manipulator.

Grasping and manipulation is the final output of the end-effector and the goal of the robot.

Manipulation requires understanding of [1] contact kinematics [2] forces applied through contacts [3] dynamics of rigid bodies.

Old grasping and manipulation used contact kinematics and contact force modeling to solve manipulation.

given d(q) between bodies with d(q) > 0 means no contact, d(q) = 0 means contact and d(q) < 0 means penetration.

![Screenshot 2024-10-30 at 1.56.06 PM.png](../../images/Screenshot_2024-10-30_at_1.56.06_PM.png)

The contact normal vector determines the forces/properties of the contact.

Contacts can be rolling, sliding, or breaking free.

- The **active constraint** means that $\dot{d} = 0$, guaranteeing that the contact stays connected to the object
- The **impenetrability constraint** means that $\dot{d} \geq 0$ meaning that the contact doesn’t penetrate the object
- The **rolling constraint** determines if the objects will move relative to each other. If a chance in the effector results in no relative change to the object, it’s rolling, otherwise it’s sliding.

Different contacts based on the constraints in movement they add to the manipulated rigid body limit the degrees of freedom of that bodies motion.

**Form closure** of a rigid body is achieved if a set of stationary objects prevents all motion of the body. If this is provided by robot fingers, it’s called **form closure grasp**.

We can use a **grasp metric** to evaluate the quality of the grasp, where it has to be contacting/not penetrating, and then we can further evaluate it’s quality with other metrics.

The contacts result in a **force closure** if the composite wrench cone contains the entire wrench space.

> For any point contact and contact label, the number of equality constraints of the body’s motion caused by that contact is equal to the number of wrench freedoms it provides.
> 

Manipulation involves anything where manipulators impose motions or forces with the purpose of achieving the motion or restraint of objects.

To manipulate objects, we combine kinematic constraints, Coulomb friction law, and the dynamics of rigid bodies.

A **dynamic grasp** uses inertial forces to keep the object pressed against the fingers while the fingers move.

3 ingredients determine rigid-body contact interactions:

1. the contact kinematics that describe feasible motions for rigid bodies in contact
2. contact force model that describes forces transmitted through frictional contacts
3. rigid-body dynamics

# Notes: Model Control


This paper goes over the general modeling problem that robotics has to solve, focused on kinematics and dynamics models.

Classical robotics uses models that are based on human insight, resembling manual feature engineering in early deep learning.

This paper focuses on how model based approaches developed directly from classical robotics, using learned models to replace the analytical models manually programmed using robotic kinematics and dynamics.

This paper preceded the success of deep reinforcement learning and generative models for model learning, but sets the stage for these successes.

> While classical robotics relies on manually generated models that are based on human insights into physics, future autonomous, cognitive robots need to be able to automatically generate models that are based on information which is extracted from the data streams accessible to the robot.

Purely classical analytical models that use human knowledge alone to create models leave much functionality to be desired.

This is why model learning is a necessary tool in robotics.

> It is essential to study how the basic, underlying mechanisms of the world can be learned. This approach is known as _model learning._

This is the approach that allowed deep learning into robotics. Rather than manually modeling the world, we use systems to learn world models that can perform actions.

> The reason for this increasing interest [in model learning for robot control] is that accurate analytical models are often hard to obtain due to the complexity of modern robot systems and their presence in unstructured, uncertain and human-inhabited environments

> Model learning can be a useful alternative to manual pre-programming, as the model is estimated directly from measured data.

Before model learning, all robotic control systems were manually programmed for every edge case, which required complex analytic physics models.

> Unknown non-linearities can be directly taken in account, while they are neglected by the standard physics-based modeling techniques and by hand-crafted models.

Due to the complexity of the real world, man made physics models are prone to large errors which make analytical modeling ineffective.

> In the context of control, a model contains essential information about the system and describes the influence of an agent on this system.

### 1. Problem Statement

Model learning specific deals with learning a state transition model and an output model based on the current state and action and a noise function:

$$
s_{k+1} = f(s_k, a_k) + \epsilon_f \\
y_k = h(s_k, a_k) + \epsilon_y
$$

Specifically model learning deals with learning a model of the world to predict the next state $s_{k+1}$ given the current state $s_k$ and action $a_k$ (where the underlying world dynamics determines the true function) and a function to predict the output $y_k$ given the state and action.

> Due to the increasing complexity of robot systems, analytical models are more difficult to obtain. This problem leads to a variety of model estimation techniques which allow the roboticist to acquire models from data.

> In early days of adaptive control, models are learned by fitting open parameters of pre-defined parametric models.

An early naive approach to model learning used parametric analytical models that mostly define physical rules but allow room for certain variables to adjust actual predictions.

Then, real robotic or environment data could be used to tune these parameters to specific values.

However, these parametric models tended to violate consistency constraints in the original models and were prone to inaccuracy,

> Nonparametric model learning methods can avoid many of these problems. Modern nonparametric model learning approaches do not pre-define a fixed model structure but adapt the model structure to the data complexity.

This rigidity of parametric models led experimentation with nonparametric models which could fully learn from observations without being constrained to ineffective parameterization. This is the path that eventually led to current state of the art robotics.

### 2. Model Learning

> The agent has to consider two major issues. First, it needs to deduce the behavior of the system from some observed quantities. Second, having inferred this information, it needs to determine how to manipulate the system.

The first step requires inferring some unknown quantities given some observations to predict how the real world model will act given certain states and actions.

Different model types depend on different observed quantities: forward models, inverse models, mixed models, and multi-step prediction models.

The second challenge deals with learning control architectures based on the models, which can involve direct modeling, indirect modeling, and distal teacher learning.

**2.1 Model Types**

![Screenshot 2024-12-02 at 2.27.23 PM.png](../../images/Screenshot_2024-12-02_at_2.27.23_PM.png)

Given different pieces of information, we can use different model types. Given the current state $s_k$ and the intended action $a_k$, a **forward model** predicts $s_{k+1}$.

An **inverse model** predicts $a_k$ given state $s_k$ and desired state $s_{k+1}$. Most robotics control uses an inverse model, starting with high level planning to come up with an intended objective and then working backwards to obtain a specific action.

**Mixed models** can use both forward and inverse models, and **multi-step prediction models** operate over multiple time steps.

> Given a model that provides predictions of the missing information, actions can be generated by the agent. Thus, a policy is understood as a decision for controlling the system, while a model reflects the behavior of the system.

> [The forward model] expresses the physical properties of the system, the forward model represents a causal relationship between states and actions.

**2.2 Learning Architectures**

![Screenshot 2024-12-02 at 2.28.11 PM.png](../../images/Screenshot_2024-12-02_at_2.28.11_PM.png)

In direct modeling, we try to learn a model of the input to output mapping itself. For problems where the input and output are not sufficiently directly linked, like in inverse learning problems, indirect modeling can be used to train the model using a particular error measure. Additionally, we can use a forward model to train an inverse model using distal teacher learning.
